In [8]:
from sklearn.metrics import classification_report, confusion_matrix
import time
from torch._C import ParameterDict
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import PreTrainedTokenizerFast
import torch.nn.functional as F
import torch.nn as nn
import torch
import seaborn as sns
import numpy as np
import pandas as pd
import re
from torch.nn import Parameter
import math
from matplotlib import pyplot as plt
from sklearn.metrics import *


train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

dev_size = int(train_df.shape[0] * 0.10)

train_df_cpy = train_df[dev_size:]
dev_df_cpy = train_df[:dev_size]
test_df_cpy = test_df

max_length = 128
hidden_size = 128
tokenizer = None
batch_size = 32
n_epochs = 20
embed_size = 100
lr = 0.01
model_path = "bert.pt"
use_gpu = True
num_labels = 7

tokenizer = PreTrainedTokenizerFast.from_pretrained('hfl/chinese-bert-wwm')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
adj = torch.Tensor(np.identity(num_labels))


def prepare_set(dataset, max_length=max_length):
    """returns input_ids, input_masks, labels for set of data ready in BERT format"""
    global tokenizer

    input_ids = dataset
#     for i in tqdm(dataset):
#         input_ids.append(camel_case_split(i))
    tokenized = tokenizer.batch_encode_plus(input_ids, return_token_type_ids=False, return_attention_mask=False,
                                            pad_to_max_length=True, truncation=True, max_length=max_length)["input_ids"]
    return tokenized


print("preprocessing training data...")
X_train = prepare_set(train_df_cpy['exercise_text'].values.tolist())

print("preprocessing training data...")
X_dev = prepare_set(dev_df_cpy['exercise_text'].values.tolist())

print("preprocessing test data...")
# -1 labels mean that those lines were not used for the scoring

X_test = prepare_set(test_df['exercise_text'].values.tolist())

cols_target = train_df.columns[1:].tolist()

keywords = {
    "函数奇偶性": "奇函数偶函数奇偶",
    "三角函数": "正弦余弦三角函数",
    "逻辑与命题关系": "命题充分必要充要",
    "集合": "集合并集交集子集空集韦恩图",
    "导数": "导数切线极值单调递单调区间",
    "平面向量": "向量",
    "数列": "数列"
}

kcols_target = [keywords[k] for k in cols_target]


device = torch.device('cuda' if torch.cuda.is_available()
                      and use_gpu else 'cpu')

labels_pre = prepare_set(cols_target)

labels_bemb = torch.tensor(labels_pre, dtype=torch.long).to(device)
klabels_bemb = torch.tensor(prepare_set(
    kcols_target), dtype=torch.long).to(device)

y_train = train_df_cpy[cols_target].values  # 0,0,1,0,0,1,0
y_dev = dev_df_cpy[cols_target].values  # 0,0,1,0,0,1,0
y_test = test_df[cols_target].values  # 0,0,1,0,0,1,0


x_train_torch = torch.tensor(X_train, dtype=torch.long).to(
    device)  # bert(exercisetext)
x_dev_torch = torch.tensor(X_dev, dtype=torch.long).to(device)
x_test_torch = torch.tensor(X_test, dtype=torch.long).to(device)

# y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train, dtype=torch.float).to(device)
y_dev_torch = torch.tensor(y_dev, dtype=torch.float).to(device)
# y_val_torch = torch.tensor(np.hstack([y_val, y_aux_val]), dtype=torch.float32).to(device)
y_test_torch = torch.tensor(y_test, dtype=torch.float).to(device)


train_data = TensorDataset(x_train_torch, y_train_torch)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(
    train_data, sampler=train_sampler, batch_size=batch_size)


# Create the DataLoader for dev set
dev_data = TensorDataset(x_dev_torch, y_dev_torch)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(
    dev_data, sampler=dev_sampler, batch_size=batch_size)

# Create the DataLoader for dev set.
test_data = TensorDataset(x_test_torch, y_test_torch)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(
    test_data, sampler=test_sampler, batch_size=batch_size)

adj = torch.Tensor(np.identity(num_labels))

class fastText(nn.Module):

    def __init__(self, n_classes=num_labels, vocab_size=tokenizer.vocab_size, embeddings=None, emb_size=embed_size, fine_tune=True, hidden_size=hidden_size):
        super(fastText, self).__init__()

        # embedding layer
        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.set_embeddings(embeddings, fine_tune)

        # hidden layer
        self.hidden = nn.Linear(emb_size, hidden_size)
        
        # output layer
        self.fc = nn.Linear(hidden_size, n_classes)
    

    '''
    set weights of embedding layer

    input param:
        embeddings: word embeddings
        fine_tune: allow fine-tuning of embedding layer? 
                   (only makes sense when using pre-trained embeddings)
    '''
    def set_embeddings(self, embeddings, fine_tune = True):
        if embeddings is None:
            # initialize embedding layer with the uniform distribution
            self.embeddings.weight.data.uniform_(-0.1, 0.1)
        else:
            # initialize embedding layer with pre-trained embeddings
            self.embeddings.weight = nn.Parameter(embeddings, requires_grad = fine_tune)

    
    '''
    input param:
        text: input data (batch_size, word_pad_len)
        words_per_sentence: sentence lengths (batch_size)

    return: 
        scores: class scores (batch_size, n_classes)
    '''
    def forward(self, text, words_per_sentence):
        # word embedding
        embeddings = self.embeddings(text) # (batch_size, word_pad_len, emb_size)
        
        # average word embeddings in to sentence erpresentations
        avg_embeddings = embeddings.mean(dim = 1).squeeze(1) # (batch_size, emb_size)
        hidden = self.hidden(avg_embeddings) # (batch_size, hidden_size)
        
        # compute probability
        scores = self.fc(hidden) # (batch_size, n_classes)
        
        return scores , None

class Model(nn.Module):
    def __init__(self, hidden_size, embed_size, max_features, num_classes, max_length):
        super().__init__()
        self.embedding = nn.Embedding(max_features, embed_size)
        self.fc = nn.Linear(max_length, max_length*2)
        self.fc2 = nn.Linear(max_length*2, num_classes)


    def forward(self, x, step_len):
        weights = torch.randn(1,1)
        aux_result  = self.fc(x.float())

        aux_result  = self.fc2(aux_result)
        aux_result = F.sigmoid(aux_result)
        return aux_result, weights


def train_model(model, loss_fn, lr=0.001, batch_size=32, n_epochs=10, max_length=64):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda epoch: 0.6 ** epoch)

    training_loss = []
    validation_loss = []
    training_acc = []
    validation_acc = []
    best_loss = float("inf")

    for epoch in range(n_epochs):
        start_time = time.time()
        training_res = []
        validataion_res = []
        model.train()
        avg_loss = 0
        y_train_preds = []
        y_train_batchs = []
        y_valid_preds = []
        y_valid_batchs = []
        for data in tqdm(train_dataloader, disable=False):
            x_batch = data[:-1]
            y_batch = data[-1]
            y_batch_labels = y_batch.detach().cpu().numpy()
#             if y_trues.shape[0] == 1:
#                 y_trues = y_batch_labels
#             else:
#                 print(y_trues, y_batch_labels)
#                 y_trues = np.concatenate(y_trues, np.array(y_batch_labels))
            y_train_batchs += y_batch_labels.tolist()
            y_pred, _ = model(*x_batch, max_length)
            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)

#             if y_preds.shape[0] == 1:
#                 y_preds = y_pred_labels
#             else:
#                 y_preds = np.concatenate(y_preds, y_pred_labels)

            y_train_preds += y_pred_labels.tolist()
            loss = nn.BCEWithLogitsLoss()(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_dataloader)
        tacc = accuracy_score(y_train_batchs, y_train_preds)
        training_acc.append(tacc)
        training_loss.append(avg_loss)
        model.eval()
        print(f'... Validating ... ')
        avg_val_loss = 0
        for val_data in tqdm(dev_dataloader, disable=False):
            x_batch = val_data[:-1]
            y_batch = val_data[-1]
            y_batch_labels = y_batch.detach().cpu().numpy()
            y_valid_batchs += y_batch_labels.tolist()
#             if y_trues_v.shape[0] == 1:
#                 y_trues_v = y_batch_labels
#             else:
#                 y_trues_v = np.concatenate(y_trues_v, y_batch_labels)
            y_pred, _ = model(*x_batch, max_length)
            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)
            y_valid_preds += y_pred_labels.tolist()
#             if y_preds_v.shape[0] == 1:
#                 y_preds_v = y_pred_labels
#             else:
#                 y_preds_v = np.concatenate(y_preds_v, y_pred_labels)

            val_loss = nn.BCEWithLogitsLoss()(y_pred, y_batch)
            avg_val_loss += val_loss.item() / len(dev_dataloader)
        vacc = accuracy_score(y_valid_batchs, y_valid_preds)
        validation_acc.append(vacc)
        elapsed_time = time.time() - start_time
        validation_loss.append(avg_val_loss)
        if avg_val_loss < best_loss:
            print('saving the best model so far')
            best_loss = avg_val_loss
            torch.save(model.state_dict(), model_path)
        print(f'Epoch {epoch + 1}/{n_epochs}\t training_loss={avg_loss:.4f} \t validation_loss={avg_val_loss: 4f} \t train_epoch_acc={tacc: 4f} \t valid_epoch_acc={vacc: 4f} \t time={elapsed_time:.2f}s')
        scheduler.step()
    return training_loss, validation_loss, training_acc, validation_acc


def evaluate(model):
    # Create the DataLoader for dev set.
    model.eval()
    preds = np.zeros((1, num_labels))
    y_test_preds = []
    y_test_trues = []
    with torch.no_grad():
        for tst_data in tqdm(test_dataloader, disable=False):
            x_batch = tst_data[:-1]
            y_batch_labels = tst_data[-1].detach().cpu().numpy()
            y_test_trues += y_batch_labels.tolist()
            y_pred, _ = model(*x_batch, max_length)

            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)
            y_test_preds += y_pred_labels.tolist()
            correct_labels = (y_pred_labels == y_batch_labels)
            preds += correct_labels.sum(axis=0)

    return preds, y_test_preds, y_test_trues


model = fastText()
model.to(device)
trainloss, vloss, training_acc, validation_acc = train_model(model=model, loss_fn=None, lr=lr, batch_size=batch_size,
                                                             n_epochs=n_epochs, max_length=max_length)

true_positives, y_test_preds, y_test_trues = evaluate(model)
for i, acc in enumerate((true_positives / test_df.shape[0])[0]):
    print(f"{cols_target[i]} accuracy is {acc*100:.2f}%")


preprocessing training data...
preprocessing training data...
preprocessing test data...
/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2068: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  8%|▊         | 7/84 [00:00<00:01, 68.54it/s]... Validating ... 
saving the best model so far
Epoch 1/20	 training_loss=0.3060 	 validation_loss= 0.156287 	 train_epoch_acc= 0.318657 	 valid_epoch_acc= 0.666667 	 time=1.29s
  8%|▊         | 7/84 [00:00<00:01, 68.32it/s]... Validating ... 
saving the best model so far
Epoch 2/20	 training_loss=0.1054 	 vali

In [11]:
t = classification_report(y_test_trues, y_test_preds, target_names=cols_target,digits=3)
tacc = accuracy_score(y_test_trues, y_test_preds)
print(t,tacc)


              precision    recall  f1-score   support

        三角函数      0.800     0.387     0.522        31
       函数奇偶性      0.944     0.893     0.918       187
          导数      0.927     0.919     0.923       247
        平面向量      0.975     0.956     0.965       204
          数列      0.979     0.967     0.973       243
     逻辑与命题关系      0.945     0.861     0.901       180
          集合      0.963     0.578     0.722        45

   micro avg      0.952     0.894     0.922      1137
   macro avg      0.933     0.794     0.846      1137
weighted avg      0.950     0.894     0.918      1137
 samples avg      0.929     0.917     0.916      1137
 0.8538306451612904


In [4]:
class fastText(nn.Module):

    def __init__(self, n_classes, vocab_size, embeddings, emb_size, fine_tune, hidden_size):
        super(fastText, self).__init__()

        # embedding layer
        self.embeddings = nn.Embedding(vocab_size, emb_size)
        self.set_embeddings(embeddings, fine_tune)

        # hidden layer
        self.hidden = nn.Linear(emb_size, hidden_size)
        
        # output layer
        self.fc = nn.Linear(hidden_size, n_classes)
    

    '''
    set weights of embedding layer

    input param:
        embeddings: word embeddings
        fine_tune: allow fine-tuning of embedding layer? 
                   (only makes sense when using pre-trained embeddings)
    '''
    def set_embeddings(self, embeddings, fine_tune = True):
        if embeddings is None:
            # initialize embedding layer with the uniform distribution
            self.embeddings.weight.data.uniform_(-0.1, 0.1)
        else:
            # initialize embedding layer with pre-trained embeddings
            self.embeddings.weight = nn.Parameter(embeddings, requires_grad = fine_tune)

    
    '''
    input param:
        text: input data (batch_size, word_pad_len)
        words_per_sentence: sentence lengths (batch_size)

    return: 
        scores: class scores (batch_size, n_classes)
    '''
    def forward(self, text, words_per_sentence):
        # word embedding
        embeddings = self.embeddings(text) # (batch_size, word_pad_len, emb_size)
        
        # average word embeddings in to sentence erpresentations
        avg_embeddings = embeddings.mean(dim = 1).squeeze(1) # (batch_size, emb_size)
        hidden = self.hidden(avg_embeddings) # (batch_size, hidden_size)
        
        # compute probability
        scores = self.fc(hidden) # (batch_size, n_classes)
        
        return scores

In [16]:
class TextCNN1D(nn.Module):
    def __init__(self, n_classes=num_labels, vocab_size=tokenizer.vocab_size, embeddings=None, emb_size=embed_size, fine_tune=True, 
                 n_kernels=4, kernel_sizes=(2,2), dropout=0.5, n_channels = 1):

        super(TextCNN1D, self).__init__()

        # embedding layer
        self.embedding1 = nn.Embedding(vocab_size, emb_size)
        self.set_embeddings(embeddings, 1, fine_tune)

        if n_channels == 2:
            # multichannel: a static channel and a non-static channel
            # which means embedding2 is frozen
            self.embedding2 = nn.Embedding(vocab_size, emb_size)
            self.set_embeddings(embeddings, 1, False)
        else:
            self.embedding2 = None

        # 1d conv layer
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels = n_channels, 
                out_channels = n_kernels, 
                kernel_size = size * emb_size,
                stride = emb_size
            ) 
            for size in kernel_sizes
        ])

        self.fc = nn.Linear(len(kernel_sizes) * n_kernels, n_classes) 
        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()


    '''
    set weights of embedding layer

    input param:
        embeddings: word embeddings
        layer_id: embedding layer 1 or 2 (when adopting multichannel architecture)
        fine_tune: allow fine-tuning of embedding layer? 
                   (only makes sense when using pre-trained embeddings)
    '''
    def set_embeddings(self, embeddings, layer_id = 1, fine_tune = True):
        if embeddings is None:
            # initialize embedding layer with the uniform distribution
            if layer_id == 1:
                self.embedding1.weight.data.uniform_(-0.1, 0.1)
            else:
                self.embedding2.weight.data.uniform_(-0.1, 0.1)
        else:
            # initialize embedding layer with pre-trained embeddings
            if layer_id == 1:
                self.embedding1.weight = nn.Parameter(embeddings, requires_grad = fine_tune)
            else:
                self.embedding2.weight = nn.Parameter(embeddings, requires_grad = fine_tune)


    '''
    input param:
        text: input data (batch_size, word_pad_len)
        words_per_sentence: sentence lengths (batch_size)

    return: 
        scores: class scores (batch_size, n_classes)
    '''
    def forward(self, text, words_per_sentence):

        batch_size = text.size(0)

        # word embedding
        embeddings = self.embedding1(text).view(batch_size, 1, -1)  # (batch_size, 1, word_pad_len * emb_size)
        # multichannel
        if self.embedding2:
            embeddings2 = self.embedding2(text).view(batch_size, 1, -1)  # (batch_size, 1, word_pad_len * emb_size)
            embeddings = torch.cat((embeddings, embeddings2), dim = 1) # (batch_size, 2, word_pad_len * emb_size)

        # conv
        conved = [self.relu(conv(embeddings)) for conv in self.convs]  # [(batch size, n_kernels, word_pad_len - kernel_sizes[n] + 1)]

        # pooling
        pooled = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in conved]  # [(batch size, n_kernels)]
        
        # flatten
        flattened = self.dropout(torch.cat(pooled, dim = 1))  # (batch size, n_kernels * len(kernel_sizes))
        scores = self.fc(flattened)  # (batch size, n_classes)
        
        return scores, None

In [22]:


def train_model(model, loss_fn, lr=0.001, batch_size=32, n_epochs=10, max_length=64):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer, lambda epoch: 0.6 ** epoch)

    training_loss = []
    validation_loss = []
    training_acc = []
    validation_acc = []
    best_loss = float("inf")

    for epoch in range(n_epochs):
        start_time = time.time()
        training_res = []
        validataion_res = []
        model.train()
        avg_loss = 0
        y_train_preds = []
        y_train_batchs = []
        y_valid_preds = []
        y_valid_batchs = []
        for data in tqdm(train_dataloader, disable=False):
            x_batch = data[:-1]
            y_batch = data[-1]
            y_batch_labels = y_batch.detach().cpu().numpy()
#             if y_trues.shape[0] == 1:
#                 y_trues = y_batch_labels
#             else:
#                 print(y_trues, y_batch_labels)
#                 y_trues = np.concatenate(y_trues, np.array(y_batch_labels))
            y_train_batchs += y_batch_labels.tolist()
            y_pred, _ = model(*x_batch, max_length)
            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)

#             if y_preds.shape[0] == 1:
#                 y_preds = y_pred_labels
#             else:
#                 y_preds = np.concatenate(y_preds, y_pred_labels)

            y_train_preds += y_pred_labels.tolist()
            loss = nn.BCEWithLogitsLoss()(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss += loss.item() / len(train_dataloader)
        tacc = accuracy_score(y_train_batchs, y_train_preds)
        training_acc.append(tacc)
        training_loss.append(avg_loss)
        model.eval()
        print(f'... Validating ... ')
        avg_val_loss = 0
        for val_data in tqdm(dev_dataloader, disable=False):
            x_batch = val_data[:-1]
            y_batch = val_data[-1]
            y_batch_labels = y_batch.detach().cpu().numpy()
            y_valid_batchs += y_batch_labels.tolist()
#             if y_trues_v.shape[0] == 1:
#                 y_trues_v = y_batch_labels
#             else:
#                 y_trues_v = np.concatenate(y_trues_v, y_batch_labels)
            y_pred, _ = model(*x_batch, max_length)
            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)
            y_valid_preds += y_pred_labels.tolist()
#             if y_preds_v.shape[0] == 1:
#                 y_preds_v = y_pred_labels
#             else:
#                 y_preds_v = np.concatenate(y_preds_v, y_pred_labels)

            val_loss = nn.BCEWithLogitsLoss()(y_pred, y_batch)
            avg_val_loss += val_loss.item() / len(dev_dataloader)
        vacc = accuracy_score(y_valid_batchs, y_valid_preds)
        validation_acc.append(vacc)
        elapsed_time = time.time() - start_time
        validation_loss.append(avg_val_loss)
        if avg_val_loss < best_loss:
            print('saving the best model so far')
            best_loss = avg_val_loss
            torch.save(model.state_dict(), model_path)
        print(f'Epoch {epoch + 1}/{n_epochs}\t training_loss={avg_loss:.4f} \t validation_loss={avg_val_loss: 4f} \t train_epoch_acc={tacc: 4f} \t valid_epoch_acc={vacc: 4f} \t time={elapsed_time:.2f}s')
        scheduler.step()
    return training_loss, validation_loss, training_acc, validation_acc


def evaluate(model):
    # Create the DataLoader for dev set.
    model.eval()
    preds = np.zeros((1, num_labels))
    y_test_preds = []
    y_test_trues = []
    with torch.no_grad():
        for tst_data in tqdm(test_dataloader, disable=False):
            x_batch = tst_data[:-1]
            y_batch_labels = tst_data[-1].detach().cpu().numpy()
            y_test_trues += y_batch_labels.tolist()
            y_pred, _ = model(*x_batch, max_length)

            y_pred_labels = (torch.sigmoid(
                y_pred).detach().cpu().numpy() > 0.5)
            y_test_preds += y_pred_labels.tolist()
            correct_labels = (y_pred_labels == y_batch_labels)
            preds += correct_labels.sum(axis=0)

    return preds, y_test_preds, y_test_trues

cnntmodel = TextCNN1D(n_kernels=5, kernel_sizes=(5,64), dropout=0.2)
cnntmodel.to(device)
trainloss, vloss, training_acc, validation_acc = train_model(model=cnntmodel, loss_fn=None, lr=0.01, batch_size=batch_size,
                                                             n_epochs=n_epochs, max_length=max_length)

  2%|▏         | 2/84 [00:00<00:04, 17.53it/s]... Validating ... 
saving the best model so far
Epoch 1/20	 training_loss=0.2539 	 validation_loss= 0.110788 	 train_epoch_acc= 0.504104 	 valid_epoch_acc= 0.838384 	 time=4.86s
  2%|▏         | 2/84 [00:00<00:04, 17.39it/s]... Validating ... 
saving the best model so far
Epoch 2/20	 training_loss=0.1251 	 validation_loss= 0.105951 	 train_epoch_acc= 0.772388 	 valid_epoch_acc= 0.848485 	 time=4.83s
  2%|▏         | 2/84 [00:00<00:04, 17.48it/s]... Validating ... 
Epoch 3/20	 training_loss=0.1095 	 validation_loss= 0.116843 	 train_epoch_acc= 0.787687 	 valid_epoch_acc= 0.861953 	 time=4.81s
  2%|▏         | 2/84 [00:00<00:04, 17.42it/s]... Validating ... 
saving the best model so far
Epoch 4/20	 training_loss=0.0899 	 validation_loss= 0.096292 	 train_epoch_acc= 0.828731 	 valid_epoch_acc= 0.851852 	 time=4.81s
  2%|▏         | 2/84 [00:00<00:04, 17.23it/s]... Validating ... 
saving the best model so far
Epoch 5/20	 training_loss=0.0873 	

In [25]:
true_positives, y_test_preds, y_test_trues = evaluate(cnntmodel)
for i, acc in enumerate((true_positives / test_df.shape[0])[0]):
    print(f"{cols_target[i]} accuracy is {acc*100:.2f}%")
t = classification_report(y_test_trues, y_test_preds, target_names=cols_target,digits=3)
tacc = accuracy_score(y_test_trues, y_test_preds)
print(t,tacc)


100%|██████████| 31/31 [00:00<00:00, 164.55it/s]
/Users/maywzh/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
三角函数 accuracy is 96.88%
函数奇偶性 accuracy is 97.48%
导数 accuracy is 95.56%
平面向量 accuracy is 98.89%
数列 accuracy is 99.09%
逻辑与命题关系 accuracy is 98.08%
集合 accuracy is 97.38%
              precision    recall  f1-score   support

        三角函数      0.000     0.000     0.000        31
       函数奇偶性      0.976     0.888     0.930       187
          导数      0.939     0.879     0.908       247
        平面向量      0.995     0.951     0.972       204
          数列      0.992     0.971     0.981       243
     逻辑与命题关系      0.994     0.900     0.945       180
          集合      1.000     0.422     0.594        45

   micro avg   